#Conectamos a google drive e importamos librerias

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
import pandas as pd
!pip install pandas_summary
from pandas_summary import DataFrameSummary
import numpy as np
import datetime
!pip install isoweek
from isoweek import Week
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
!pip install sklearn_pandas
from sklearn_pandas import DataFrameMapper

from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, Input, Flatten, Concatenate, Dense, BatchNormalization, Activation, LeakyReLU, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical


# Para importar el Helper
%cd "/gdrive/Shareddrives/Redes neuronales/TP2"
!pip install import-ipynb
import import_ipynb
from Helper import *

/gdrive/Shareddrives/Redes neuronales/TP2
importing Jupyter notebook from Helper.ipynb


# Cargamos los archivos con el preprocesamiento completo

In [3]:
df = pd.read_feather('data preprecesada/train_normalized_data_!sales0.fth')
df_test = pd.read_feather('data preprecesada/test_normalized_data.fth')

# Embeddings

Vemos las dimensiones de entrada al modelo para cada variables:

In [4]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']

In [5]:
uniques = DataFrameSummary(df[cat_vars]).summary().loc[['uniques']]
uniques

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,PromoInterval,CompetitionOpenSinceYear,Promo2SinceYear,State,Week,Events,Promo_fw,Promo_bw,StateHoliday_bool_fw,StateHoliday_bool_bw,SchoolHoliday_fw,SchoolHoliday_bw
uniques,1115,7,3,12,31,4,25,26,4,3,4,21,8,12,52,22,6,6,3,3,8,8


En el archivo TP2-Modelo esta parte es realizada por una función en el Helper llamada: **get_cat_vars_model**

Acá se va a detallar en profundidan la función de esta capa y lo que se estaría obteniendo en el modelo. Recordando que solo se hace este paso para las variables categoricas, se asigna la dimensionalidad de las diferentes capas para cada variable que se va a usar para predecir.

In [6]:
cat_var_dict = {'Store': 50, 'DayOfWeek': 4, 'Year': 2, 'Month': 6,
'Day': 10, 'StateHoliday': 3, 'CompetitionMonthsOpen': 2,
'Promo2Weeks': 1, 'StoreType': 2, 'Assortment': 3, 'PromoInterval': 3,
'CompetitionOpenSinceYear': 4, 'Promo2SinceYear': 4, 'State': 6,
'Week': 2, 'Events': 4, 'Promo_fw': 1,
'Promo_bw': 1, 'StateHoliday_fw': 1,
'StateHoliday_bw': 1, 'SchoolHoliday_fw': 1,
'SchoolHoliday_bw': 1}

Acá se están definiendo las dimensiones de salida de las capas. Esto quiere decir que, por ejemplo, para 'DayOfWeek' que según 'uniques' puede tomar 7 valores posibles, el embedding los agrupa en 4 clusters.

## Explicación función creado de Embeddings

En la función **get_car_vars_model** primero se crea el Input de la capa Embedding:

In [7]:
# Ejemplo con dos variables:
day_of_week_input = Input(shape=(1,), name="day_of_week_input")
store_input = Input(shape=(1,), name="store_input")

Descripción Input:

Input(_Cantidad de variables categoricas que van a ser la entrada a ese embedding_, _nombre_)

Luego se crea la capa de Embedding:

In [8]:
day_of_week_Embedd_out = Embedding(uniques['DayOfWeek'][0], cat_var_dict['DayOfWeek'], name="day_of_week_Embedd")(day_of_week_input)
store_Embedd_out = Embedding(uniques['Store'][0], cat_var_dict['Store'], name="store_Embedd")(store_input)

Descripción de función Embedding:

Embedding(_Cantidad de valores de entrada_, _Cantidad de clusters de salida_, _Nombre de la capa_)(_Input_)

In [9]:
# A modo de ejemplo se visualizan las entradas y salidas de los embeddings anteriores:
print(f'Entradas: {day_of_week_input.shape}, {uniques["DayOfWeek"][0]}, {store_input.shape}, {uniques["Store"][0]}')
print(f'Salidas: {day_of_week_Embedd_out.shape}, {store_Embedd_out.shape}')

Entradas: (None, 1), 7, (None, 1), 1115
Salidas: (None, 1, 4), (None, 1, 50)


Se puede ver que a la dimensionalidad de la salida de cada capa son las elegidas en el diccionario cat_var_dict

Luego en la función **get_cat_vars_model** se aplica la función Flatten:



In [10]:
day_of_week_Embedd_flat = Flatten(name="day_of_week_flat")(day_of_week_Embedd_out)
store_Embedd_flat = Flatten(name="store_flat")(store_Embedd_out)

In [11]:
print(f'Entradas: {day_of_week_Embedd_out.shape}, {store_Embedd_out.shape}')
print(f'Salidas: {day_of_week_Embedd_flat.shape}, {store_Embedd_flat.shape}')

Entradas: (None, 1, 4), (None, 1, 50)
Salidas: (None, 4), (None, 50)


Aplanar un tensor significa eliminar todas las dimensiones excepto una.

Una capa Flatten en Keras cambia la forma del tensor para que tenga una forma que sea igual al número de elementos contenidos en el tensor. Es equivalente a pasar una matrix de salida de la función de Embedding a un array de 4 y 50 elementos respectivamente.

Finalmente la función devolvera la lista de los Inputs y la lista de las capas de Embedding con el reshape adecuado.

# Capas funciones continuas

En este caso no se hace una capa de Embeddings ya que se estaría trabajando con variables continuas, por eso la función **get_cont_vars_input** en el Helper, va a devolver solo los Inputs armados para cada variable continua. 

Usamos de ejemplo la variable correspondiente a la variable de distancia con el competidor. Como antes, dado que la entrada es solo una variable, la dimensión de entrada es 1.

In [12]:
DataFrameSummary(df[['CompetitionDistance']]).summary().loc[['uniques','types']]

,CompetitionDistance
uniques,654
types,numeric


In [13]:
competition_distance_input = Input(shape=(1,), name="competition_distance_input")

Esta va a ser la salida de la función usada en el modelo completo.

# Armado de modelo

Para esta parte se usan las funciones: **MLP_layers** y **crear_MLP**

Primero se juntan todos los inputs en una sola capa de entrada:

In [14]:
merged = Concatenate(name='All_Concatenate')([day_of_week_Embedd_flat, store_Embedd_flat, competition_distance_input])

**Concatenate**: Capa que concatena una lista de entradas.
Toma como entrada una lista de tensores, todos de la misma forma excepto por el eje de concatenación, y devuelve un solo tensor que es la concatenación de todas las entradas.

Luego se crean y concatenan las diferentes capas del modelo:

In [15]:
x = Dense(1000, activation='relu')(merged) # Capa oculta
x = Dense(500, activation='relu')(x) # Capa oculta

Hasta acá se crea con la función **MLP_layers** las diferentes capas ocultas. Luego, con la función **crear_MLP** estaremos creando las capas de salida y dandole las propiedades de modelo.

La cantidad de capas de salida se definen con el parametro add_customers (explicado con más detalle en el script TP2-Modelo)

In [16]:
add_customers = False
# Creamos el output de Sales:
output_1 = Dense(1, name='Sales', activation='linear')(x)
# El modelo se crea en base a si también queremos predecir customers:
if add_customers:
  output_2 = Dense(1, name='Customers', activation='linear')(x)
  model = Model([day_of_week_input, store_input, competition_distance_input], [output_1, output_2])
else: 
  model = Model([day_of_week_input, store_input, competition_distance_input], [output_1])


Descripción función de modelo:

**MODEL**(Inputs, Capas Outputs)

In [17]:
#model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
day_of_week_input (InputLayer)  [(None, 1)]          0                                            
__________________________________________________________________________________________________
store_input (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
day_of_week_Embedd (Embedding)  (None, 1, 4)         28          day_of_week_input[0][0]          
__________________________________________________________________________________________________
store_Embedd (Embedding)        (None, 1, 50)        55750       store_input[0][0]                
______________________________________________________________________________________________





**Model**: Va a agrupar las capas creadas en un objeto con funciones de entrenamiento e inferencia.

En **summary** se puede ver como están concatenadas las diferentes partes del modelo.

# Entrenamiento y obtención del modelo

## Obtención del conjunto de entrenamiento y validación

Primero se divide en el conjunto de entrenamiento, tomando los registros con fechas menores al 1/7/2015, y de validación, con fechas mayores al 1/7/2015.

In [18]:
df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]

Vemos la cantidad de registros que hay en el conjunto de validación junto con el porcentaje que representa el conjunto de validación de los datos totales:


In [19]:
print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_val)/(len(df_train) + len(df_val)+len(df_test))}')

Cantidad en val: 30188, porcentaje: 0.034094322958666226


##Entrenamiento del modelo

Primero, con la función **obtener_x_y**, se preparan los dataset de train, validación y test para entrar a la función de entrenamiento.

En esta función, primero, se divide el dataset con las variables elegidas en 3 columnas (1 por cada variable) para entrenar el modelo. Para esto se utiliza la función de numpy **hsplit**:

In [39]:
X_train = np.hsplit(df_train[['DayOfWeek', 'Store', 'CompetitionDistance']].values, 3)
X_val = np.hsplit(df_val[['DayOfWeek', 'Store', 'CompetitionDistance']].values, 3)
X_test = np.hsplit(df_test[all_vars].values, len(all_vars))
print(f'({len(X_val)},{len(X_val[0])})')

(3,30188)


Se puede ver como la divide en 3 columnas de 30188 de largo (cantidad de registros en el dataset de validación).

Luego, al igual que explicamos anteriormente, a la función se le ingresa una variable booleana 'add_customers' que define cuales seran las salidas

In [21]:
  if add_customers:
      y_out_columns = ['Sales', 'Customers']
  else:
      y_out_columns = ['Sales']

Con el parámetro 'log_output' se define si se desea normalizar y pasar a una escala logarítmica los valores de salida (True) o si se desean normalizar por la media y varianza (False). En este caso se muestra la normalización por la media y la varianza:

In [22]:
log_output = False
if log_output:
  # Escala logaritmica
  max_log_y = np.max(np.log(df['Sales']))
  y_train = np.log(df['Sales'])/max_log_y
  y_val = np.log(df_val[y_out_columns].values)/max_log_y
else: 
  # Normalizacion
  y_mean = df_train['Sales'].mean()
  y_std = df_train['Sales'].std()
  y_max = df_train[y_out_columns].max().values
  
  y_train = (df_train['Sales'].values - y_mean)/y_std
  y_val = (df_val['Sales'].values - y_mean)/y_std

Luego, se entrena el modelo utilizando la función **entrenar_MLP**. 

Primero, con esta esta función se configura el modelo para el entrenamiento. Con el parámetro *optimizer* se define la función del optimizador, en este caso, se elige el Adam optimizer que implementa el algoritmo de Adam para actualizar los parámetros de peso para minimizar la función de costo. 

Se elige que se pueda variar el parámetro *lr*, learning_rate o tasa de aprendizaje. Es un hiperparámetro que controla cuanto cambiar el modelo en respuesta al error estimado cada vez que se actualizan los pesos del modelo. En general, tiene un valor positivo pequeño entre 0 y 1. Controla la rapidez con la que el modelo se adapta al problema. Las tasas de aprendizaje más pequeñas requieren más epochs de entrenamiento dados los cambios más pequeños realizados en los pesos en cada actualización, mientras que las tasas de aprendizaje más grandes dan como resultado cambios rápidos y requieren menos epochs de entrenamiento.
Una lr demasiado grande puede hacer que el modelo converja demasiado rápido a una solución subóptima, mientras que una lr demasiado pequeño puede hacer que el proceso se atasque.

Los *epochs* son la cantidad de ciclos o iteraciones de aprendizaje durante el entrenamiento de la red. Este es el numero de veces que se ejecutaran los algoritmos de forwardpropagation y backpropagation. En cada ciclo (epoch) todos los datos de entrenamiento pasan por la red neuronal para que esta aprenda sobre ellos. Un número chico de epoch no  es suficiente para lograr una respuesta óptima y hay desajuste pero si se aumenta mucho, también aumenta el número de actualizaciones de peso en la red neuronal y la curva cambia de desajuste a sobreajuste.

El *batch_size* es el numero de datos que tiene cada iteración de un ciclo (epoch), es la cantidad de lotes o muestras que se propagarán a través de la red. Si es menor a la cantidad de muestras, el procedimiento de entrenamiento, en general, requiere menos memoria. Sin embargo, hay que tener cuidado ya que cuanto más pequeño sea el lote, menos precisa será la estimación del gradiente

*metrics* es la lista de métricas que el modelo evaluará durante el entrenamiento y las pruebas. en este caso, se elige el error cuadrático medio o mse.


*loss* es la función de costo o loss function. Es la encargada de cuantificar la distancia entre el valor real y el valor predicho por la red, en otras palabras, mide cuánto se equivoca la red al realizar predicciones. En la mayoría de casos, la función de costo devuelve valores positivos. Cuanto más próximo a cero es el resultado de la función, mejor son las predicciones de la red (menor error), siendo cero cuando las predicciones se corresponden exactamente con el valor real. En este caso, se utiliza mse o error cuadrático medio que suele ser la más utilizada en problemas de regresión. 

*verbose* es un parámetro utilizado para ver el progreso de entrenamiento para cada epoch. En este caso, definiéndolo como 2, se muestra la progresión mostrando el número de epoch por el que va. 

In [23]:
lr=0.001
epochs=1
batch_size=256

metrics=['mse']
loss='mse'

verbose = 2

model.compile(optimizer=Adam(learning_rate=lr), metrics=metrics, loss=loss)

Descripción model.compile:

model.compile(optimizador, métricas, loss function)

Esta función configura el modelo para ser entrenado.

In [24]:
model_chosen = '/gdrive/Shareddrives/Redes neuronales/TP2/modelos generados/modelo_detalle_funciones.hdf5'

if add_customers:
    checkpoint = ModelCheckpoint(model_chosen, monitor='val_Sales_mse', verbose=2, save_best_only=True)
else:
    checkpoint = ModelCheckpoint(model_chosen, monitor='val_loss', verbose=2, save_best_only=True)

Descripción ModelCheckpoint:

ModelCheckpoint(nombre del archivo con que se guarda el mejor modelo, metrica a monitorear, verbose, save_best_only)

Esta función se usa junto con el entrenamiento usando model.fit () para guardar un modelo o pesos (en un archivo de punto de control) en algún intervalo, por lo que el modelo o los pesos se pueden cargar más tarde para continuar el entrenamiento desde el estado guardado.

Con el parametro *add_customers* estamos diciendole a la función ModelCheckpoint cual va a ser la metrica a monitorear para decidir el mejor modelo. 

Definiendo *save_best_only* = True solo guarda cuando el modelo se considera el "mejor" y no se sobrescribe el último mejor modelo de acuerdo con la cantidad monitoreada. Considera un modelo como el "mejor" cuando la métrica que se está monitoreando es la más óptima, en este caso, cuando el error es lo más cercano a cero. 

Descripción if(add_customers):

En el caso que *add_customers* sea TRUE nos queremos fijar solo la evolución de las ventas, ya que la predicción de los custumers solo se utiliza como parámetro de regularización.

En caso que sea FALSE, no se tiene una predicción de Custumers entonces directamente se puede fijar en la métrica loss. 

En ambos casos se evalua para el conjunto de validación.

Por último, se entrena el modelo:

In [25]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),  epochs=1, batch_size=256, callbacks=[checkpoint], verbose=verbose)

3181/3181 - 57s - loss: 0.3350 - mse: 0.3350 - val_loss: 0.2398 - val_mse: 0.2398

Epoch 00001: val_loss improved from inf to 0.23980, saving model to /gdrive/Shareddrives/Redes neuronales/TP2/modelos generados/modelo_detalle_funciones.hdf5


Descripción model.fit:

model.fit(train, test,validation data, epochs, batch size,callbacks,epochs)

Esta función entrena el modelo para la cantidad de epochs dados. 

##Obtención de métricas del modelo

### Métricas con hold out validation

Para obtener las métricas se utiliza la función **metricas_MLP**.  

En esta función, primero se compila el modelo con *model.compile* y se cargan los pesos con *model.load_weigths* con el modelo guardado anteriormente como ya explicamos.

In [26]:
model.compile(optimizer=Adam(learning_rate=lr), metrics=metrics, loss=loss)
model.load_weights(model_chosen)

Luego, evalúo el modelo en los datos de validación con model.evaluate.

In [27]:
res_metrics = model.evaluate(X_val, y_val,return_dict = True)
res_metrics

944/944 [==============================] - 2s 2ms/step - loss: 0.2398 - mse: 0.2398


{'loss': 0.23979893326759338, 'mse': 0.23979893326759338}

La función te devuelve el loss y las métricas (*metrics*) defininidas anteriormente, en este caso, el mse (mean squared error) del modelo para el dataset de validación.

### Métricas con cross validation

Para chequear que las métricas obtenidas sean confiables, se obtienen las métricas utilizando Time series cross validation. Para esto se utiliza la función **cross_val_score_MLP**.



In [32]:
metrics_cross = []
k = 5
all_vars = ['DayOfWeek','Store'] + ['CompetitionDistance']

for i in range(8-k,8):
  df_cross_train = df[df.Date < datetime.datetime(2015, i, 1)]  
  df_cross_val = df[df.Date >= datetime.datetime(2015, i, 1)]
  df_cross_val = df_cross_val[df.Date < datetime.datetime(2015, i+1, 1)]

  print(f'Fechas en df_train:, {min(df_cross_train.Date)}, - {max(df_cross_train.Date)}')
  print(f'Fechas en df_val:, {min(df_cross_val.Date)}, - {max(df_cross_val.Date)}')

  X_cross_train,X_cross_val,X_cross_test,y_cross_train,y_cross_val,y_norm = obtener_x_y(df_cross_train,df_cross_val,df_test,all_vars,add_customers=add_customers,log_output=log_output)  

  model_chosen=f'modelos generados/bestmodel_detallefciones_add_customers_{add_customers}_log_output_{log_output}_lr_{lr}_batch_size_{batch_size}_crossval_{i}.hdf5'

  history,model_cross = entrenar_MLP(X_cross_train,X_cross_val,y_cross_train,y_cross_val,all_vars,model=model,add_customers=add_customers,log_output=log_output,lr=lr,metrics=metrics,loss=loss,model_chosen=model_chosen,epochs = epochs,batch_size = batch_size,verbose=verbose)  

  metric_cross = metricas_MLP(X_cross_val,y_cross_val,model_cross,model_chosen=model_chosen)    
  metrics_cross.append(metric_cross)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Fechas en df_train:, 2013-01-01 00:00:00, - 2015-02-28 00:00:00
Fechas en df_val:, 2015-03-01 00:00:00, - 2015-03-31 00:00:00
2750/2750 - 48s - loss: 0.0021 - mse: 0.0021 - val_loss: 0.0019 - val_mse: 0.0019

Epoch 00001: val_loss improved from inf to 0.00190, saving model to modelos generados/bestmodel_detallefciones_add_customers_False_log_output_False_lr_0.001_batch_size_256_crossval_3.hdf5
909/909 [==============================] - 2s 2ms/step - loss: 0.0019 - mse: 0.0019 - rmspe: 0.2153
Fechas en df_train:, 2013-01-01 00:00:00, - 2015-03-31 00:00:00
Fechas en df_val:, 2015-04-01 00:00:00, - 2015-04-30 00:00:00
2863/2863 - 49s - loss: 0.0021 - mse: 0.0021 - val_loss: 0.0028 - val_mse: 0.0028

Epoch 00001: val_loss improved from inf to 0.00285, saving model to modelos generados/bestmodel_detallefciones_add_customers_False_log_output_False_lr_0.001_batch_size_256_crossval_4.hdf5
842/842 [==============================] - 2s 2ms/step - loss: 0.0028 - mse: 0.0028 - rmspe: 0.2346
Fechas

In [36]:
metrics_cross

[{'loss': 0.001901597366668284,
  'mse': 0.001901597366668284,
  'rmspe': 0.2152891457080841},
 {'loss': 0.002846181159839034,
  'mse': 0.002846181159839034,
  'rmspe': 0.2346181571483612},
 {'loss': 0.0013640486868098378,
  'mse': 0.0013640486868098378,
  'rmspe': 0.17443738877773285},
 {'loss': 0.002334670862182975,
  'mse': 0.002334670862182975,
  'rmspe': 0.2293253093957901},
 {'loss': 0.0013034248258918524,
  'mse': 0.0013034248258918524,
  'rmspe': 0.21060769259929657}]

Primero se define la cantidad de veces que se desea validar el modelo con el parámetro *k*. Definido en 5, este parámetro hace que se utilizen 5 datasets de entrenamiento y validación distintos. 

Luego se utilizan las funciones explicadas anteriormente: **obtener_x_y** para la preparación y normalización de datasets, **entrenar_MLP** para entrenar el modelo y **metricas_MLP** para obtener las métricas.

Con model_chosen, se guardan los modelos entrenados y metrics_cross guarda todas las metricas obtenidas.

##Obtención de predicciones con el modelo

Para obtener las predicciones se usa la función **predic_MLP**.

En esta función, primero se compila el modelo con *model.compile* y se cargan los pesos con *model.load_weigths* con el modelo guardado anteriormente como ya explicamos.

In [37]:
model.compile(optimizer=Adam(learning_rate=lr), metrics=metrics, loss=loss)
model.load_weights(model_chosen)

Luego, a través del parámetro *log_output* elegimos el tipo de normalización para normalización para la salida. 

In [40]:
model.compile(optimizer=Adam(learning_rate=lr), metrics=metrics, loss=loss)
model.load_weights(model_chosen)

if log_output:
  max_log_y = y_norm[0]
  if add_customers:
      y_pred_test = np.exp(model.predict(X_test, verbose=1)[0][:, 0]*max_log_y[0])
  else:
      y_pred_test = np.exp(model.predict(X_test, verbose=1)*max_log_y)[:,0]
else:
  y_mean = y_norm[0]
  y_std = y_norm[1]
  y_max = y_norm[2]
  if add_customers:
      y_pred_test = (model.predict(X_test, verbose=1)[0]*y_std[0] + y_mean[0])[:,0]
  else:

      y_pred_test = model.predict(X_test, verbose=1)[:,0]*y_max

1284/1284 [==============================] - 3s 2ms/step




Con model.predict se obtienen las predicciones para el dataset de validación y testeo. Se elige verbose = 1 para poder hacer el seguimiento del testeo.

Descripción model.predict:

model.predict(X_test, verbose=1)

Esta función predice el resultado del dataset de X_test. 

Luego, al igual que explicamos la normalización a través del *log_output*, se elige el tipo de normalización y se normaliza la salida del model.predict, la predicción del dataset de testeo hecha con el modelo. En este caso, mostramos como se normaliza por la media y la varianza ya que log_output = False.

In [41]:
y_pred_test[df_test['Open'] == 0] = 0

Para todos los datos en que la tienda está cerrada (open = 0) se predice que las compras van a ser cero por lo que se reemplaza la predicción de ventas por cero.

In [42]:
y_pred_test

array([ 4425.63223723,  7199.97959822,  8600.03610232, ...,
        5803.16067208, 21923.74859995,  6676.63926877])